# Parameters parsing

By priority, parameters of the model can be parsed from three ways when initializing a model:

- Input as keyword arguments (`**kwargs`).
- Input in the `parameters` argument as dictionary.
- Save in a `.yaml` file, and point at its path as the `settings_file` argument.

## Basic parameters

Here is an basic example of `example.yaml`:

```yaml
demo:
    testing: 'from file'

nature:
    param1: 0.1

human:
    arg1: 5
```

In [1]:
from abses import MainModel

path = "example.yaml"
parameters = {'testing': 'from dictionary',}

model = MainModel(
    name='demo',  # should be same as the name in `.yaml`
    base='tests',
    parameters=parameters,
    testing='from argument',
    settings_file=path
)

# show parameters
model.params

{'testing': 'from argument'}

We passed both 'testing' parameter in three ways, but the one "from argument" has priority.

Note that when setting a parameter you need to specify at first level which module it belongs to (model, nature, human)... Only parameter names that can be found in the configuration file can be passed in by the other two methods without specifying a module.

In [2]:
# only `arg1` will be parsed because it exists in the config file.
model = MainModel(name='demo', base='tests', parameters={'arg1': 1, 'arg2': 2}, settings_file=path)

model.nature.params

Unsolved parameters: ['arg2'].


{'param1': 0.1}

`arg2` cannot be parsed because it's not sure which module should use this parameter.

## Special keywords

Some keywords of parameters are associated with behavior of the model.

- `record`: The variables to be recorded in each step.
- `report`: The variables to be reported after.
- `model: steps`: how many steps to simulate.
- `model: seed`: random seed for all random simulations.

Now, let's set a variable "random_value" in the human module that will automatically record changes, to show how these special parameters work.

In [3]:
from abses import BaseHuman
from abses.actor import perception


class Human(BaseHuman):
    args = ['arg1']

    @perception
    def random_value(self):
        return self.model.random.random()


parameters = {
    'demo': {'steps': 5, 'seed': 1},
    'human': {'arg1': 2, 'record': 'random_value'}
}


model = MainModel(
    human_class=Human,
    name='demo',
    base='bases',
    settings_file=path,
    parameters=parameters
)
model.run()

Completed: 5 steps
Run time: 0:00:00.003634
Simulation finished


DataDict {
'info': Dictionary with 9 keys
'parameters': 
    'constants': Dictionary with 2 keys
'variables': 
    'Human': DataFrame with 1 variable and 5 rows
    'BaseNature': DataFrame with 0 variables and 1 row
'reporters': DataFrame with 1 variable and 1 row
}

You can see that the model runs five simulations after the run, thanks to the keyword parameter `steps`.

Since the "`record`" parameter specifies "`random_value`" as the record variable, its dynamic will be recorded. 

_`random_value` is a method of Human, but uses the decorator `@perception` to treat it as a variable, here._

In [4]:
# This property stores all recording variables.
model.human.recording

# The random value recorded, applied the random seed.
model.human.output

# random seed = 1
model.output.reporters

{'random_value'}

,t,random_value,obj_id
2000,0,0.763775,2
2001,1,0.255069,2
2002,2,0.495435,2
2003,3,0.449491,2
2004,4,0.651593,2


,seed
0,1


**Note** that we have added a class attribute `arg1` to our custom model class, which indicates that the module must require an argument input named '`arg1`'. We have overwritten the '`arg1`' in the congif settings file with parameters when creating the model, (you can see that it has a value of $2$, not $1$ as in the `yaml` file)

**Model would have reported an error if it had not been entered.**

In [5]:
# Required parameters `arg1` of human.
model.human.arg1

2

## Spatial settings

 The keyword `world` under '`Nature`' module defines the spatial environmental behavior. There are two ways to generate an artificial world:

 1. [Use real geographic data (`.shp`, `.tif`, ...)](/nature/geodata.ipynb).
 2. Create a simple rectangular world by specific parameters.

- The first approach simply requires the world parameter to be **the data path**.
- Here, we give another simple demonstration of the second approach.

![7jzXmY](https://songshgeo-picgo-1302043007.cos.ap-beijing.myqcloud.com/uPic/7jzXmY.jpg)

As shown above, by default, the nature module is created as a 9*9 world, setting the parameters of such a world as:

```yaml
## world.yaml
width: 9
height: 9
outside: 1
border: 1
resolution: 10
```

## Time settings

The keyword `time` under `model` module defines the time simulating behavior.

By default these parameters are used:

```yaml
start: "2000-01-01 00:00:00"
end: None
freq: "Y"
```

These parameters mean that:
1. The simulation starts at 1th, Jan., 2000.
2. Simulation has no default termination time.
3. Each step of the simulation represents a year.

`ABSESpy` uses [`pandas.Period`](https://pandas.pydata.org/docs/reference/api/pandas.Period.html#pandas.Period) to manage time, where for more information on how to create a time you can refer.

It means, if you want to simulate at `Y` frequency level, `start = '2000'` is overall as same as `'2000-01-01'` ...